# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe3782cbeb0>
├── 'a' --> tensor([[ 1.5792, -1.2659, -0.0574],
│                   [-1.1537,  0.4559, -1.4980]])
└── 'x' --> <FastTreeValue 0x7fe3782cb550>
    └── 'c' --> tensor([[ 0.7273,  1.4121, -0.4545, -0.4050],
                        [-0.0544,  1.1466, -0.6517,  0.6267],
                        [ 0.2717, -1.3464,  0.4923,  0.1415]])

In [4]:
t.a

tensor([[ 1.5792, -1.2659, -0.0574],
        [-1.1537,  0.4559, -1.4980]])

In [5]:
%timeit t.a

76 ns ± 1.1 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe3782cbeb0>
├── 'a' --> tensor([[ 0.7178,  1.1009, -0.5209],
│                   [-0.9275,  0.1518,  1.7561]])
└── 'x' --> <FastTreeValue 0x7fe3782cb550>
    └── 'c' --> tensor([[ 0.7273,  1.4121, -0.4545, -0.4050],
                        [-0.0544,  1.1466, -0.6517,  0.6267],
                        [ 0.2717, -1.3464,  0.4923,  0.1415]])

In [7]:
%timeit t.a = new_value

79 ns ± 0.819 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.5792, -1.2659, -0.0574],
               [-1.1537,  0.4559, -1.4980]]),
    x: Batch(
           c: tensor([[ 0.7273,  1.4121, -0.4545, -0.4050],
                      [-0.0544,  1.1466, -0.6517,  0.6267],
                      [ 0.2717, -1.3464,  0.4923,  0.1415]]),
       ),
)

In [10]:
b.a

tensor([[ 1.5792, -1.2659, -0.0574],
        [-1.1537,  0.4559, -1.4980]])

In [11]:
%timeit b.a

64.4 ns ± 0.58 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2396,  1.2687,  0.3151],
               [ 1.5872, -1.2582,  0.0917]]),
    x: Batch(
           c: tensor([[ 0.7273,  1.4121, -0.4545, -0.4050],
                      [-0.0544,  1.1466, -0.6517,  0.6267],
                      [ 0.2717, -1.3464,  0.4923,  0.1415]]),
       ),
)

In [13]:
%timeit b.a = new_value

606 ns ± 5.66 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.04 µs ± 11.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.5 µs ± 182 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

285 µs ± 16.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

285 µs ± 17.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe378215040>
├── 'a' --> tensor([[[ 1.5792, -1.2659, -0.0574],
│                    [-1.1537,  0.4559, -1.4980]],
│           
│                   [[ 1.5792, -1.2659, -0.0574],
│                    [-1.1537,  0.4559, -1.4980]],
│           
│                   [[ 1.5792, -1.2659, -0.0574],
│                    [-1.1537,  0.4559, -1.4980]],
│           
│                   [[ 1.5792, -1.2659, -0.0574],
│                    [-1.1537,  0.4559, -1.4980]],
│           
│                   [[ 1.5792, -1.2659, -0.0574],
│                    [-1.1537,  0.4559, -1.4980]],
│           
│                   [[ 1.5792, -1.2659, -0.0574],
│                    [-1.1537,  0.4559, -1.4980]],
│           
│                   [[ 1.5792, -1.2659, -0.0574],
│                    [-1.1537,  0.4559, -1.4980]],
│           
│                   [[ 1.5792, -1.2659, -0.0574],
│                    [-1.1537,  0.4559, -1.4980]]])
└── 'x' --> <FastTreeValue 0x7fe2ce5388e0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

45.9 µs ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe37825f6a0>
├── 'a' --> tensor([[ 1.5792, -1.2659, -0.0574],
│                   [-1.1537,  0.4559, -1.4980],
│                   [ 1.5792, -1.2659, -0.0574],
│                   [-1.1537,  0.4559, -1.4980],
│                   [ 1.5792, -1.2659, -0.0574],
│                   [-1.1537,  0.4559, -1.4980],
│                   [ 1.5792, -1.2659, -0.0574],
│                   [-1.1537,  0.4559, -1.4980],
│                   [ 1.5792, -1.2659, -0.0574],
│                   [-1.1537,  0.4559, -1.4980],
│                   [ 1.5792, -1.2659, -0.0574],
│                   [-1.1537,  0.4559, -1.4980],
│                   [ 1.5792, -1.2659, -0.0574],
│                   [-1.1537,  0.4559, -1.4980],
│                   [ 1.5792, -1.2659, -0.0574],
│                   [-1.1537,  0.4559, -1.4980]])
└── 'x' --> <FastTreeValue 0x7fe2cdff7400>
    └── 'c' --> tensor([[ 0.7273,  1.4121, -0.4545, -0.4050],
                        [-0.0544,  1.1466, -0.6517,  0.6267],
                 

In [23]:
%timeit t_cat(trees)

40.3 µs ± 544 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

84.7 µs ± 4.36 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.5792, -1.2659, -0.0574],
                [-1.1537,  0.4559, -1.4980]],
       
               [[ 1.5792, -1.2659, -0.0574],
                [-1.1537,  0.4559, -1.4980]],
       
               [[ 1.5792, -1.2659, -0.0574],
                [-1.1537,  0.4559, -1.4980]],
       
               [[ 1.5792, -1.2659, -0.0574],
                [-1.1537,  0.4559, -1.4980]],
       
               [[ 1.5792, -1.2659, -0.0574],
                [-1.1537,  0.4559, -1.4980]],
       
               [[ 1.5792, -1.2659, -0.0574],
                [-1.1537,  0.4559, -1.4980]],
       
               [[ 1.5792, -1.2659, -0.0574],
                [-1.1537,  0.4559, -1.4980]],
       
               [[ 1.5792, -1.2659, -0.0574],
                [-1.1537,  0.4559, -1.4980]]]),
    x: Batch(
           c: tensor([[[ 0.7273,  1.4121, -0.4545, -0.4050],
                       [-0.0544,  1.1466, -0.6517,  0.6267],
                       [ 0.2717, -1.3464,  0.4923,  0.1415]],
         

In [26]:
%timeit Batch.stack(batches)

104 µs ± 1.44 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.5792, -1.2659, -0.0574],
               [-1.1537,  0.4559, -1.4980],
               [ 1.5792, -1.2659, -0.0574],
               [-1.1537,  0.4559, -1.4980],
               [ 1.5792, -1.2659, -0.0574],
               [-1.1537,  0.4559, -1.4980],
               [ 1.5792, -1.2659, -0.0574],
               [-1.1537,  0.4559, -1.4980],
               [ 1.5792, -1.2659, -0.0574],
               [-1.1537,  0.4559, -1.4980],
               [ 1.5792, -1.2659, -0.0574],
               [-1.1537,  0.4559, -1.4980],
               [ 1.5792, -1.2659, -0.0574],
               [-1.1537,  0.4559, -1.4980],
               [ 1.5792, -1.2659, -0.0574],
               [-1.1537,  0.4559, -1.4980]]),
    x: Batch(
           c: tensor([[ 0.7273,  1.4121, -0.4545, -0.4050],
                      [-0.0544,  1.1466, -0.6517,  0.6267],
                      [ 0.2717, -1.3464,  0.4923,  0.1415],
                      [ 0.7273,  1.4121, -0.4545, -0.4050],
                      [-0.0544,  

In [28]:
%timeit Batch.cat(batches)

185 µs ± 3.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

617 µs ± 27.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
